In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from htorch import layers
import time

In [2]:
device = torch.device('cpu')

data = torch.tensor(pd.read_csv("../data/mnist/train.csv", header=None).values)
x, y = (data[:, 1:]/255).float().to(device), torch.nn.functional.one_hot(data[:, 0].long(), 10).to(device)

test = torch.tensor(pd.read_csv("../data/mnist/test.csv", header=None).values)
x_test, y_test = (test[:, 1:]/255).float().to(device), torch.nn.functional.one_hot(test[:, 0].long(), 10).to(device)

In [3]:
class Quat(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = layers.QLinear(196, 75)
        self.fc2 = layers.QLinear(75, 25)
        self.fc3 = nn.Linear(100, 10)

    def forward(self, x):
        # x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
model = Quat()
model.to(device)
optimiser = torch.optim.Adam(model.parameters(), lr=1.2e-3)
losses = []
trainacc = []
testacc =[]

scale = 0.025253813613805267
img_mat = tensor([[ 0.2487, -0.7623,  0.4338,  ...,  0.9388,  0.7033, -0.7972],
        [ 0.6684, -0.0946, -0.1756,  ..., -0.9971, -0.6412,  0.4621],
        [ 0.6433,  0.9782,  0.6268,  ..., -0.6094,  0.0790, -0.3066],
        ...,
        [-0.3207,  0.1750,  0.7024,  ...,  0.5215, -0.6097, -0.7213],
        [ 0.0964, -0.2245,  0.5541,  ..., -0.5793,  0.8000, -0.5365],
        [-0.6700, -0.1994, -0.5265,  ...,  0.0184, -0.9387,  0.5279]])
r.shape = torch.Size([196, 75]) i.shape = torch.Size([196, 75]) j.shape = torch.Size([196, 75]) k.shape = torch.Size([196, 75])
scale = 0.040824829046386304
img_mat = tensor([[ 0.9424,  0.9341,  0.5188,  ..., -0.1208,  0.2544, -0.0768],
        [-0.0553,  0.5031,  0.9847,  ..., -0.1914,  0.7617, -0.5287],
        [ 0.0708,  0.7721,  0.8971,  ..., -0.9540,  0.6323, -0.9205],
        ...,
        [-0.8344, -0.1185, -0.3041,  ..., -0.4586, -0.4662,  0.8545],
        [-0.6151, -0.5624,  0.4542,  ...,  0.5084,  0.5734,  0.7341

/home/aritra/project/quartLT23/htorch/quaternion.py:491: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:350.)
  if self.dim() == 1:
/home/aritra/project/quartLT23/htorch/quaternion.py:494: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:350.)
  out = self.q[:, :self.shape[1] // 4]
/home/aritra/project/quartLT23/htorch/quaternion.py:500: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:350.)
  if self.dim() == 1:
/home/aritra/project/quartLT23/htorch/quaternion.py:504: UserWarning: Defining your `_

In [4]:
t_f = 0
t_b = 0

for epoch in tqdm(range(1000)):
    batch_size = 2**13
    for i in range(0, len(x), batch_size):
        batch_x, batch_y = x[i:i+batch_size], y[i:i+batch_size].float()
        optimiser.zero_grad()
        t0 = time.time()
        output = model(batch_x)
        t_f += time.time() - t0
        t0 = time.time()
        loss = F.mse_loss(output, batch_y)
        loss.backward()
        t_b += time.time() - t0
        optimiser.step()
    losses.append(loss.item())

100%|██████████| 1000/1000 [01:59<00:00,  8.38it/s]


In [5]:
t_f, t_b

(41.9013946056366, 60.65592360496521)

In [6]:
# for epoch in tqdm(range(80)):
#     batch_size = 60
#     for i in range(0, len(x), batch_size):
#         batch_x, batch_y = x[i:i+batch_size], y[i:i+batch_size].float()
#         optimiser.zero_grad()
#         output = model(batch_x)
#         loss = F.mse_loss(output, batch_y)
#         loss.backward()
#         optimiser.step()
#     losses.append(loss.item())
    
#     accuracy = (model(x).argmax(1) == y.argmax(1)).sum().item()/len(x)
#     trainacc.append(accuracy)
#     # print(f"Training Accuracy: {accuracy*100:.2f}%")

#     accuracy = (model(x_test).argmax(1) == y_test.argmax(1)).sum().item()/len(x_test)
#     # print(f"Testing Accuracy: {accuracy*100:.2f}%")
#     testacc.append(accuracy)

# plt.plot(range(len(trainacc)), trainacc, label='training')
# plt.plot(range(len(testacc)), testacc,label = 'testing')
# plt.legend()